In [90]:
# current focus
import torchvision
import torch


# ------------------- Get Data -------------------
# 60,000 samples, 28x28 pixels / sample
train_dataset = torchvision.datasets.MNIST(train=True, download=True, root='./')
train_rand = torch.randperm(train_dataset.data.shape[0]) # for picking random images/labels, like randint except has no repeats
train_img_tensors = train_dataset.data.reshape(-1,784)[train_rand]
Y = train_dataset.targets[train_rand]

# 10,000 samples, 28x28 pixels / sample
test_dataset = torchvision.datasets.MNIST(train=False, download=True, root='./')
test_rand = torch.randperm(test_dataset.data.shape[0]) # for picking random images/labels, like randint except has no repeats
test_img_tensors = test_dataset.data.reshape(-1,784)[test_rand]
test_Y = test_dataset.targets[test_rand]


# ------------------- Train -------------------
zero = torch.tensor(0)
batch_size = torch.tensor([32])
# input nodes batch
# 32 x 784
X = train_img_tensors[:batch_size] / 255

h1_nodes = 128
w1 = torch.randn((h1_nodes, 784))
w1 = w1 / h1_nodes**0.5
l1 = X@w1.T
relu1 = torch.maximum(zero, l1)

h2_nodes = 64
w2 = torch.randn((h2_nodes, h1_nodes))
w2 = w2 / h2_nodes**0.5
l2 = relu1@w2.T
relu2 = torch.maximum(zero, l2)

out_nodes = 10
w3 = torch.randn((out_nodes, h2_nodes))
# w3[:,1] += 10 # manually adjusts weights
w3 = w3 / out_nodes**0.5
logits = relu2@w3.T

e_to_the_logits = torch.e**logits
probs = e_to_the_logits / torch.sum(e_to_the_logits, dim=1, keepdim=True) # prob = softmax. softmax is an activation function, as is relu

# ------------------- Loss -------------------

# understand this before moving on to anything else
loss = -torch.log(probs[range(batch_size), Y[:batch_size]]).mean()
loss

tensor(3.7376)

In [91]:
a = -torch.log(probs[range(batch_size), Y[:batch_size]])[Y[0]]
a

tensor(4.3177)

In [94]:
b = -torch.log(probs[range(batch_size), Y[:batch_size]])[Y[0]]
# probs[0,Y[0]] -= 0.03
b

tensor(4.3177)

In [96]:
probs[range(batch_size), Y[:batch_size]], -torch.log(probs[range(batch_size), Y[:batch_size]])

(tensor([0.0149, 0.0133, 0.0098, 0.0610, 0.0080, 0.0145, 0.0093, 0.0667, 0.0354,
         0.0197, 0.1773, 0.0008, 0.0019, 0.0399, 0.3319, 0.0189, 0.0049, 0.1446,
         0.0263, 0.0033, 0.0072, 0.1003, 0.0192, 0.0022, 0.0323, 0.0207, 0.0085,
         0.0773, 0.0198, 0.2942, 0.0460, 0.4007]),
 tensor([4.2097, 4.3177, 4.6225, 2.7971, 4.8281, 4.2310, 4.6750, 2.7076, 3.3397,
         3.9279, 1.7301, 7.0970, 6.2552, 3.2211, 1.1030, 3.9665, 5.3128, 1.9338,
         3.6364, 5.7264, 4.9350, 2.3001, 3.9513, 6.1046, 3.4331, 3.8757, 4.7688,
         2.5596, 3.9242, 1.2236, 3.0786, 0.9145]))